# **Disease Prediction Chatbot**



In [3]:
# importing all necessary libraries
import warnings
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split, cross_val_score
from statistics import mean
from nltk.corpus import wordnet 
import requests
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from itertools import combinations
from collections import Counter
import operator
import math
from sklearn.linear_model import LogisticRegression

warnings.simplefilter("ignore")

# Downloading resources required for NLTK pre-processing

In [4]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data] 

True

Here we are trying to find synonyms of the symptoms entered by the user.
This is necessary as the user may use a term for a symptom which may be different from the one present in dataset.
This improves the accuracy by reducing the wrong predictions even when symptoms for a disease are entered slightly different than the ones on which model is trained.

*Synonyms are searched on Thesaurus.com and NLTK Wordnet*

In [5]:
# utlities for pre-processing
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
splitter = RegexpTokenizer(r'\w+')

Disease Combination dataset contains the combinations for each of the disease present in dataset as practically it is often observed that it is not necessary for a person to have a disease when all the symptoms are faced by the patient or the user.



*df_comb -> Dataframe consisting of dataset generated by combining symptoms for each disease.*

*df_norm -> Dataframe consisting of dataset which contains a single row for each diseases with all the symptoms for that corresponding disease.*

**Dataset contains 261 diseases and their symptoms**

In [6]:
# Load Dataset
df_comb = pd.read_csv("dis_sym_dataset_comb.csv") # Disease combination
df_norm = pd.read_csv("dis_sym_dataset_norm.csv") # Individual Disease


In [ ]:
df_norm 

In [ ]:
df_norm.describe()

In [ ]:
df_norm.isnull().sum()

In [ ]:
df_comb.isnull().sum()

In [ ]:
df_comb

In [7]:
X = df_comb.iloc[:, 1:] # all columns except target
Y = df_comb.iloc[:, 0:1] # only target

Using **Logistic Regression (LR) Classifier** as it gives better accuracy compared to other classification models.

Cross validation is done on dataset with cv = 5

In [8]:
lr = LogisticRegression()
lr = lr.fit(X, Y)
scores = cross_val_score(lr, X, Y, cv=5)

In [9]:
X = df_norm.iloc[:, 1:]
Y = df_norm.iloc[:, 0:1]

In [10]:
# List of symptoms
dataset_symptoms = list(X.columns)

# Symptoms initially taken from user.

In [11]:
# returns the list of synonyms of the input word from thesaurus.com (https://www.thesaurus.com/) and wordnet (https://www.nltk.org/howto/wordnet.html)
def synonyms(term):
    synonyms = []
    response = requests.get('https://www.thesaurus.com/browse/{}'.format(term))
    soup = BeautifulSoup(response.content,  "html.parser") 
    try:
        container=soup.find('section', {'class': 'MainContentContainer'}) 
        row=container.find('div',{'class':'css-191l5o0-ClassicContentCard'})
        row = row.find_all('li')# finding children tags in html
        for x in row:
            synonyms.append(x.get_text())
    except:
        None
    for syn in wordnet.synsets(term):
        synonyms+=syn.lemma_names()
    return set(synonyms)

In [12]:
# Taking symptoms from user as input 
user_symptoms = str(input("Please enter symptoms separated by comma(,):\n")).lower().split(',')
# Preprocessing the input symptoms
processed_user_symptoms=[]
for sym in user_symptoms:
    sym=sym.strip()
    sym=sym.replace('-',' ')
    sym=sym.replace("'",'')
    sym = ' '.join([lemmatizer.lemmatize(word) for word in splitter.tokenize(sym)])
    processed_user_symptoms.append(sym)
print(processed_user_symptoms)

Please enter symptoms separated by comma(,):
fever, Coughing, Stomach Pain
['fever', 'coughing', 'stomach pain']


Pre-processing on symptoms entered by user is done.

In [13]:
# Taking each user symptom and finding all its synonyms and appending it to the pre-processed symptom string
user_symptoms = [] # empty list for user symptoms
for user_sym in processed_user_symptoms:
    user_sym = user_sym.split() # splitting user symptoms
    str_sym = set()
    for comb in range(1, len(user_sym)+1):
        for subset in combinations(user_sym, comb):
            subset=' '.join(subset)
            subset = synonyms(subset) 
            str_sym.update(subset) 
    str_sym.add(' '.join(user_sym))
    user_symptoms.append(' '.join(str_sym).replace('_',' '))
# query expansion performed by joining synonyms found for each symptoms initially entered
print("After query expansion done by using the symptoms entered")
print(user_symptoms)

After query expansion done by using the symptoms entered
['febrility feverishness fever pyrexia febricity', 'coughing cough', 'ail suffer breadbasket venter support pain sensation annoyance infliction abide put up hurting digest bother abdomen brook pain painful sensation botheration stomach hurt tum stick out bear pain in the neck pain in the ass endure anguish nuisance stomach pain belly tolerate painfulness trouble tummy stand']


The below procedure is performed in order to show the symptom synonmys found for the symptoms entered by the user.

The symptom synonyms and user symptoms are matched with the symptoms present in dataset. Only the symptoms which matches the symptoms present in dataset are shown back to the user. 

In [16]:
# Loop over all the symptoms in dataset and check its similarity score to the synonym string of the user-input 
# symptoms. If similarity>0.5, add the symptom to the final list
found_symptoms = set()# empty set for finalized symptoms
for idx, data_sym in enumerate(dataset_symptoms):
    data_sym_split=data_sym.split()# splitting dataset symptoms
    for user_sym in user_symptoms:
        count=0
        
        for symp in data_sym_split:
            if symp in user_sym.split():
                count+=1  # if a word in user symptom split is found in data_symptom_split then count will increase
        if count/len(data_sym_split)>0.5: # finding similarity between words in both split sets if threshold if morethan 0.5 then that symptom from user_symptoms can be added to found symptoms.
            found_symptoms.add(data_sym)
found_symptoms = list(found_symptoms)

## **Prompt the user to select the relevant symptoms by entering the corresponding indices.**

In [34]:
# Print all found symptoms
print("Top matching symptoms from your search!")
for idx, symp in enumerate(found_symptoms):
    print(idx,":",symp)# what ever found symptoms will be printed here with index
    
# Show the related symptoms found in the dataset and ask user to select among them
select_list = input("\nPlease select the relevant symptoms. Enter indices (separated-space):\n").split()

# Find other relevant symptoms from the dataset based on user symptoms based on the highest co-occurance with the
# ones that is input by the user
dis_list = set()
final_symp = [] 
counter_list = []
for idx in select_list:
    symp=found_symptoms[int(idx)]
    final_symp.append(symp)
    dis_list.update(set(df_norm[df_norm[symp]==1]['label_dis']))    # if column value is 1 then updates list with disease name
for dis in dis_list:
    row = df_norm.loc[df_norm['label_dis'] == dis].values.tolist()# row contains all 0s and 1st where 0s if disease not present in the list 1 for present in the list
    row[0].pop(0) # deleting all the rows which are not present in the list
    for idx,val in enumerate(row[0]):
        if val!=0 and dataset_symptoms[idx] not in final_symp: # checking if value already exist in the list of symptons or not if not there it will update the list
            counter_list.append(dataset_symptoms[idx])


Top matching symptoms from your search!
0 : coughing
1 : trouble sensation
2 : fever
3 : neck
4 : stomach pain
5 : painful

Please select the relevant symptoms. Enter indices (separated-space):
4 0


In [35]:
counter_list

['arm',
 'back',
 'chest pain',
 'cold sweat',
 'feeling faint upon standing',
 'feeling tired',
 'jaw',
 'nausea',
 'neck',
 'shortness breath',
 'feeling tired',
 'fever',
 'headache',
 'muscle joint pain',
 'runny nose',
 'sore throat',
 'chest tightness',
 'recurring episode wheezing',
 'shortness breath']

## To find symptoms which generally co-occur, for example with symptoms like cough, headache generally happens hence they co-occur.

In [36]:
# Symptoms that co-occur with the ones selected by user              
dict_symp = dict(Counter(counter_list))
dict_symp_tup = sorted(dict_symp.items(), key=operator.itemgetter(1),reverse=True)   
print(dict_symp_tup) 

[('feeling tired', 2), ('shortness breath', 2), ('arm', 1), ('back', 1), ('chest pain', 1), ('cold sweat', 1), ('feeling faint upon standing', 1), ('jaw', 1), ('nausea', 1), ('neck', 1), ('fever', 1), ('headache', 1), ('muscle joint pain', 1), ('runny nose', 1), ('sore throat', 1), ('chest tightness', 1), ('recurring episode wheezing', 1)]


## User is presented with a list of co-occuring symptoms to select from and is performed iteratively to recommend more possible symptoms based on the similarity to the previously entered symptoms.

As the co-occuring symptoms can be in overwhelming numbers, only the top 5 are recommended to the user from which user can select the symptoms.

If user does not have any of those 5 symptoms and wants to see the next 5, he can do so by giving input as -1.

To stop the recommendation, user needs to give input as "No".

In [ ]:
# Iteratively, suggest top co-occuring symptoms to the user and ask to select the ones applicable 
found_symptoms=[]
count=0
for tup in dict_symp_tup:
    count+=1
    found_symptoms.append(tup[0])
    if count%5==0 or count==len(dict_symp_tup):
        print("\nCommon co-occuring symptoms:")
        for idx,ele in enumerate(found_symptoms):
            print(idx,":",ele)
        select_list = input("Do you have have of these symptoms? If Yes, enter the indices (space-separated), 'no' to stop, '-1' to skip:\n").lower().split();
        if select_list[0]=='no':
            break
        if select_list[0]=='-1':
            found_symptoms = [] 
            continue
        for idx in select_list:
            final_symp.append(found_symptoms[int(idx)])
        found_symptoms = [] 

In [37]:
# Create query vector based on symptoms selected by the user
print("\nFinal list of Symptoms that will be used for prediction:")
sample_x = [0 for x in range(0,len(dataset_symptoms))]

for val in final_symp:
    sample_x[dataset_symptoms.index(val)]=1
    print(val)


Final list of Symptoms that will be used for prediction:
stomach pain
coughing


Prediction of disease is done

In [39]:
# Predict disease
lr = LogisticRegression()
lr = lr.fit(X, Y)
prediction = lr.predict_proba([sample_x])

Show top k diseases and their probabilities to the user.

K in this case is 10

In [40]:
k = 10
diseases = list(set(Y['label_dis']))
diseases.sort()
topk = prediction[0].argsort()[-k:][::-1]

In [41]:
topk

array([ 35,  21, 119, 211, 193, 184, 237, 175, 224, 210], dtype=int64)

# **Showing the list of top k diseases to the user with their prediction probabilities.**

 **For getting information about the suggested treatments, user can enter the corresponding index to know more details.**

In [43]:
print(f"\nTop {k} diseases predicted based on symptoms")
topk_dict = {}
# Show top 10 highly probable disease to the user.
for idx,t in  enumerate(topk):
    match_sym=set()
    row = df_norm.loc[df_norm['label_dis'] == diseases[t]].values.tolist()

    row[0].pop(0)

    for idx,val in enumerate(row[0]):
        if val!=0:
            match_sym.add(dataset_symptoms[idx])
    prob = (len(match_sym.intersection(set(final_symp)))+1)/(len(set(final_symp))+1)
    prob *= mean(scores)
    topk_dict[t] = prob
j = 0
topk_index_mapping = {}
topk_sorted = dict(sorted(topk_dict.items(), key=lambda kv: kv[1], reverse=True))
for key in topk_sorted:
    prob = topk_sorted[key]*100
    print(str(j) + " Disease name:",diseases[key], "\tProbability:",str(round(prob, 2))+"%")
    topk_index_mapping[j] = key
    j += 1



Top 10 diseases predicted based on symptoms
0 Disease name: Brucellosis 	Probability: 59.46%
1 Disease name: Asthma 	Probability: 59.46%
2 Disease name: Influenza 	Probability: 59.46%
3 Disease name: Sarcoidosis 	Probability: 29.73%
4 Disease name: Preterm birth 	Probability: 29.73%
5 Disease name: Poliomyelitis 	Probability: 29.73%
6 Disease name: Tennis elbow 	Probability: 29.73%
7 Disease name: Osteoporosis 	Probability: 29.73%
8 Disease name: Shin splints 	Probability: 29.73%
9 Disease name: SIDS 	Probability: 29.73%


In [ ]:
score = LogisticRegression.score(lr, X, Y)
print('Test Accuracy Score', score)